In [1]:
import numpy as np
import pandas as pd

In [2]:
# Read data from the file
with open("IE506_2024_progchallenge_train.txt", "r") as file:
    lines = file.readlines()

In [3]:
# Extract main classes, subclasses, and feature-value pairs
main_classes = [line.split()[0].split(":")[1] for line in lines]

In [4]:
subclasses = [line.split()[1].split(":")[1] for line in lines]

In [5]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [6]:
# Convert main_classes and subclasses to lists of lists
main_classes = [classes.split(',') for classes in main_classes]
subclasses = [classes.split(',') for classes in subclasses]

# Encode main class labels
mlb_main = MultiLabelBinarizer()
y_train_main_encoded = mlb_main.fit_transform(main_classes)

# Encode subclass labels
mlb_sub = MultiLabelBinarizer()
y_train_sub_encoded = mlb_sub.fit_transform(subclasses)

In [7]:
indices = []
values = []
data_point_indices = []

for i, line in enumerate(lines):
    parts = line.split()
    for part in parts[2:]:
        index, value = map(float, part.split(":"))
        indices.append(index)
        values.append(value)
        data_point_indices.append(i)

In [8]:
# Convert lists to numpy arrays
indices = np.array(indices, dtype=int)
values = np.array(values)
data_point_indices = np.array(data_point_indices)

In [9]:
from PIL import Image
import numpy as np

# Determine image dimensions
num_images = len(set(data_point_indices))
max_index = max(indices)
image_size = int(max_index) + 1  # Assuming indices start from 0

# Create empty arrays to store image data
images = np.zeros((num_images, image_size))

In [10]:
# Fill image data arrays
for idx, value, data_idx in zip(indices, values, data_point_indices):
    images[data_idx, int(idx)] = value

In [11]:
# Generate random indices for the training set
train_indices = np.random.choice(200000, size=160000, replace=False)

In [12]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [13]:
# Create a boolean mask for the test set
test_mask = np.ones(200000, dtype=bool)
test_mask[train_indices] = False

In [ ]:
# Splitting the data
X_train = images[train_indices]

In [ ]:
Y_train_main = y_train_main_encoded[train_indices]
Y_train_sub = y_train_sub_encoded[train_indices]

X_test = images[test_mask]
Y_test_main = y_train_main_encoded[test_mask]
Y_test_sub = y_train_sub_encoded[test_mask]

In [ ]:
# X_train.shape,Y_train_main.shape,Y_train_sub.shape,X_test.shape,Y_test_main.shape,Y_test_sub.shape,

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression

# Convert sparse matrices to CSR format


In [ ]:
# X = csr_matrix(images)

In [ ]:
X_train_csr = csr_matrix(X_train)

In [ ]:
X_test_csr = csr_matrix(X_test)

In [ ]:
X_train_csr,X_test_csr,Y_train_main.shape,Y_train_sub.shape,Y_test_sub.shape

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

# Define hyperparameters for Logistic Regression model for main class
best_params_main = {
    'C': 1.0,               # Regularization strength
    'penalty': 'l2',        # Penalty term
    'solver': 'liblinear',  # Solver algorithm
    'class_weight': 'balanced'  # Class weight
}

# Initialize Logistic Regression model for main class with best parameters
logistic_model_main = LogisticRegression(max_iter=1000, **best_params_main)

# Wrap the logistic regression model with MultiOutputClassifier for main class and sub-classes
multi_target_logistic_main = MultiOutputClassifier(logistic_model_main)

# Fit the main class model with X_train and y_train_main_encoded
multi_target_logistic_main.fit(X, y_train_main_encoded)


In [ ]:
# Get predictions for main class
predictions_main = multi_target_logistic_main.predict(X)

In [ ]:
predictions_main

In [ ]:
X

In [ ]:
from scipy.sparse import hstack

# Concatenate predictions_main with X_train_csr
X_with_predictions = hstack((X, predictions_main))

X_with_predictions

In [ ]:
# Define hyperparameters for Logistic Regression model for sub-classes
best_params_sub = {
    'C': 1.0,               # Regularization strength
    'penalty': 'l2',        # Penalty term
    'solver': 'liblinear',  # Solver algorithm
    'class_weight': 'balanced'  # Class weight
}

# Initialize Logistic Regression model for sub-classes with best parameters
logistic_model_sub = LogisticRegression(max_iter=1000, **best_params_sub)

# Wrap the logistic regression model with MultiOutputClassifier for sub-classes
multi_target_logistic_sub = MultiOutputClassifier(logistic_model_sub)

# Fit the subclass model with X_train and y_train_sub_encoded
multi_target_logistic_sub.fit(X_with_predictions, y_train_sub_encoded)


In [ ]:


# Get predictions for subclass
predictions_sub = multi_target_logistic_sub.predict(X_with_predictions)

# Now you have predictions for both main class and subclass
# predictions_main contains predictions for the main class
# predictions_sub contains predictions for the subclass


In [ ]:
# Read data from the file
with open("IE506_2024_progchallenge_test.txt", "r") as file:
    lines = file.readlines()

In [ ]:
indices_test = []
values_test = []
data_point_indices_test = []

for i, line in enumerate(lines):
    parts = line.split()
    for part in parts[2:]:
        index, value = map(float, part.split(":"))
        indices_test.append(index)
        values_test.append(value)
        data_point_indices_test.append(i)

In [ ]:
# Convert lists to numpy arrays
indices_test = np.array(indices_test, dtype=int)
values_test = np.array(values_test)
data_point_indices_test = np.array(data_point_indices_test)

In [ ]:
from PIL import Image
import numpy as np

# Determine image dimensions
num_images = len(set(data_point_indices_test))
max_index = max(indices_test)
image_size = int(max_index) + 1  # Assuming indices start from 0

# Create empty arrays to store image data
images = np.zeros((num_images, image_size))

In [ ]:
# Fill image data arrays
for idx, value, data_idx in zip(indices_test, values_test, data_point_indices_test):
    images[data_idx, int(idx)] = value

In [ ]:
images = csr_matrix(images)

In [ ]:
# Predict for main labels
y_pred_main_images = multi_target_logistic_main.predict(images)

In [ ]:
y_pred_main_images

In [ ]:
from scipy.sparse import hstack

# Concatenate predictions_main with X_train_csr
images_with_predictions = hstack((images, y_pred_main_images))

images_with_predictions

In [ ]:
# Predict for sub labels
y_pred_sub_images = multi_target_logistic_sub.predict(images_with_predictions)

In [ ]:
# Decode predicted main class labels
y_pred_main_decoded = mlb_main.inverse_transform(y_pred_main_images)

# Decode predicted subclass labels
y_pred_sub_decoded = mlb_sub.inverse_transform(y_pred_sub_images)

In [ ]:
y_pred_main_decoded

In [ ]:
import pandas as pd

# Create a DataFrame with the predicted labels
submission_df = pd.DataFrame({
    'ID': range(len(y_pred_main_images)),  # IDs starting from 0 to N_t-1
    'M': ["['" + "','".join(map(str, labels)) + "']" for labels in y_pred_main_decoded],  # Format main labels
    'S': ["['" + "','".join(map(str, labels)) + "']" for labels in y_pred_sub_decoded],   # Format sub labels
})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)
